In [1]:
import pandas as pd

from ddeserts import read_cvap_csv

In [2]:
# read in list of charter cities
charter_cities_path = 'data/cacities/charter-cities.txt'

with open(charter_cities_path) as f:
    charter_cities = { line.strip() for line in f }

In [82]:
# load the data
places_path = 'data/census/CVAP_2015-2019_ACS_csv_files/Place.csv'

pre_filter = lambda line: 'California' in line and 'Total' in line
filter = lambda row: row['state'] == 'California' and row['geotype'] != 'cdp'

ca_city_rows = list(read_cvap_csv(places_path, pre_filter=pre_filter, filter=filter))


df = pd.DataFrame.from_records(ca_city_rows, index=['table', 'line'])
df['tot_moe'] = df['tot_moe'].astype('int')
df['adu_moe'] = df['adu_moe'].astype('int')
df['cvap_moe'] = df['cvap_moe'].astype('int')
df['cit_moe'] = df['cit_moe'].astype('int')


In [85]:
# add extra columns
df['has_charter'] = df['name'].isin(charter_cities)

df['dvap_est'] = df['adu_est'] - df['cvap_est']
df['dvap_moe'] = df['cvap_moe']  # cvap_moe is always bigger
df['dvap_pct'] = df['dvap_est'] / df['adu_est']

def moe_of_div(a, a_moe, b, b_moe):
    min_a = a - a_moe
    max_a = a + a_moe
    min_b = b - b_moe
    max_b = b + b_moe
    
    d = a / b
    min_d = min_a / max_b
    max_d = max_a / min_b
    
    return max(d - min_d, max_d - d)

def make_dvap_pct_moe(r):
    return moe_of_div(r['dvap_est'], r['dvap_moe'], r['adu_est'], r['adu_moe'])

df['dvap_pct_moe'] = df.apply(make_dvap_pct_moe, axis=1)

In [86]:
dvap = df.loc[df['has_charter']][
    ['name', 'adu_est', 'dvap_est', 'dvap_moe', 'dvap_pct', 'dvap_pct_moe']].sort_values('dvap_pct')
dvap[-20:]

name  adu_est  dvap_est  dvap_moe  dvap_pct  \
table line                                                          
Place 33346         Irvine   212955     47620      2665  0.223615   
      35400    Los Angeles  3145520    728480      7692  0.231593   
      39651   Redwood City    67430     15885      1208  0.235578   
      39742       Richmond    85930     20620      1687  0.239963   
      39105         Pomona   114115     27900      1440  0.244490   
      25546        Anaheim   265820     66135      3028  0.248796   
      28900        Compton    68500     17275      1232  0.252190   
      44630       Woodlake     4855      1355       354  0.279094   
      29849         Dinuba    15740      4445       560  0.282402   
      40925    Santa Clara   102390     29365      1615  0.286796   
      37077  Mountain View    65625     19675      1237  0.299810   
      41354        Shafter    12995      4220       591  0.324740   
      43928    Watsonville    37495     12470      1030  0.332578   
      40899      Santa Ana   243370     83035      2517  0.341188   
      26287           Bell    25885      8925       684  0.344794   
      42407      Sunnyvale   120715     42505      1746  0.352110   
      40990    Santa Maria    72750     26085      1373  0.358557   
      40340        Salinas   107810     42475      1616  0.393980   
      34984        Lindsay     8950      3665       495  0.409497   
      33788      King City     9245      5415       532  0.585722   

             dvap_pct_moe  
table line                 
Place 33346      0.014138  
      35400      0.002715  
      39651      0.020762  
      39742      0.022445  
      39105      0.014770  
      25546      0.012963  
      28900      0.021753  
      44630      0.092751  
      29849      0.044168  
      40925      0.018117  
      37077      0.022087  
      41354      0.057176  
      43928      0.033544  
      40899      0.012525  
      26287      0.035384  
      42407      0.016977  
      40990      0.023452  
      40340      0.018363  
      34984      0.080318  
      33788      0.090919

In [87]:
dvap = df.loc[df['has_charter']][
    ['name', 'adu_est', 'dvap_est', 'dvap_moe', 'dvap_pct', 'dvap_pct_moe']].sort_values('dvap_est')
dvap[-30:]

name  adu_est  dvap_est  dvap_moe  dvap_pct  \
table line                                                           
Place 28900         Compton    68500     17275      1232  0.252190   
      40795       San Mateo    82505     17400      1161  0.210896   
      36583         Modesto   158385     17825      1465  0.112542   
      37077   Mountain View    65625     19675      1237  0.299810   
      38442        Pasadena   115505     19685      1499  0.170426   
      41042      Santa Rosa   140870     20410      1570  0.144885   
      39742        Richmond    85930     20620      1687  0.239963   
      40990     Santa Maria    72750     26085      1373  0.358557   
      32618         Hayward   125155     27275      1494  0.217930   
      39105          Pomona   114115     27900      1440  0.244490   
      40457  San Bernardino   152070     28590      2030  0.188006   
      40925     Santa Clara   102390     29365      1615  0.286796   
      31903        Glendale   164850     33510      1783  0.203276   
      39937       Riverside   248645     35775      2001  0.143880   
      26040     Bakersfield   264830     36715      2388  0.138636   
      42199        Stockton   223645     36910      2160  0.165038   
      28432     Chula Vista   200870     37005      2530  0.184224   
      40340         Salinas   107810     42475      1616  0.393980   
      42407       Sunnyvale   120715     42505      1746  0.352110   
      40301      Sacramento   385200     44720      2110  0.116096   
      33346          Irvine   212955     47620      2665  0.223615   
      31617          Fresno   375795     52895      2698  0.140755   
      35309      Long Beach   362340     54060      2651  0.149197   
      37740         Oakland   340625     55755      2256  0.163684   
      25546         Anaheim   265820     66135      3028  0.248796   
      40899       Santa Ana   243370     83035      2517  0.341188   
      40587   San Francisco   757415    104965      2317  0.138583   
      40535       San Diego  1129095    157595      4637  0.139576   
      40665        San Jose   797090    164515      3765  0.206395   
      35400     Los Angeles  3145520    728480      7692  0.231593   

             dvap_pct_moe  
table line                 
Place 28900      0.021753  
      40795      0.016322  
      36583      0.010056  
      37077      0.022087  
      38442      0.014360  
      41042      0.012077  
      39742      0.022445  
      40990      0.023452  
      32618      0.013578  
      39105      0.014770  
      40457      0.015431  
      40925      0.018117  
      31903      0.012121  
      39937      0.009102  
      26040      0.010001  
      42199      0.010484  
      28432      0.013968  
      40340      0.018363  
      42407      0.016977  
      40301      0.005938  
      33346      0.014138  
      31617      0.007791  
      35309      0.008120  
      37740      0.007262  
      25546      0.012963  
      40899      0.012525  
      40587      0.003059  
      40535      0.004437  
      40665      0.005139  
      35400      0.002715